In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import resample
import pickle
import hashlib
import time  
import xgboost as xgb
from xgboost import XGBClassifier
import math

# Read Data

In [3]:
print("start")
start_time = time.time() 

data1 = pd.read_csv(r"public.csv")
data2 = pd.read_csv(r"training.csv")
public1 = pd.read_csv(r"public_processed.csv")
public2 = pd.read_csv(r"uploadfile.csv")
# public_processed_df = pd.read_csv(r"uploadfile.csv")
data = pd.concat([data1, data2])
public_processed_df = pd.concat([public1, public2])

start


In [4]:
data1.shape


(609040, 26)

In [14]:
data2.shape


(8688526, 26)

In [15]:
public_processed_df.shape

(1354321, 2)

In [16]:
# # public_processed + public
# data3 = pd.read_csv(r"uploadfile.csv")
# data3.shape

In [4]:
#把特定的資料轉換成整數
def hash_to_int(value):
    return int(hashlib.sha256(value.encode()).hexdigest(), 16)

In [5]:
data["chid"] = data["chid"].apply(hash_to_int)
data["cano"] = data["cano"].apply(hash_to_int)
data["mchno"]= data["mchno"].apply(hash_to_int)
data["acqic"]= data["acqic"].apply(hash_to_int)

In [6]:
##loctm
data['loctm_hh'] = data['loctm'].apply(lambda x: math.floor(x/10000))
data['loctm_mm'] = data['loctm'].apply(lambda x: math.floor(x/100)-math.floor(x/10000)*100)
data['loctm_ss'] = data['loctm'].apply(lambda x: math.floor(x)-math.floor(x/100)*100)
data['weekday'] = data['locdt'] % 7
data['conam3000'] = np.where(data['conam'] > 3000, 1, 0)


In [ ]:
# 選取標籤為1的資料
df_positive = data[data['label'] == 1]

In [28]:
pd.set_option('display.max_columns', 100)
data.head()

,txkey,locdt,loctm,chid,cano,contp,etymd,mchno,acqic,mcc,conam,ecfg,insfg,iterm,bnsfg,flam1,stocn,scity,stscd,ovrlt,flbmk,hcefg,csmcu,csmam,flg_3dsmk,label,loctm_hh,loctm_mm,loctm_ss,weekday,conam3000
0,b734f1d9c29c8724f7fa9e14127ce19ad4fe0246055cc4...,57,202907,7545112409221137759701891378798041262236146998...,1128851795486469584559058088078669467756395378...,5,5.0,5924423611301616588777091199737320995116083692...,9694665056880918802914193734853718239332787519...,275.0,674.0,1,0,0.0,0,677,0.0,16115.0,-1.0,0,0,6.0,70.0,677,0,0,20,29,7,1,0
1,d3f6a3abc642f90d91562f91933486d5ec3a09416117d1...,58,173408,1749076699796321748633361125960037577495789411...,7440270619593444319564453073679173143151392497...,5,4.0,7379499774345514436123639587863825271999308346...,1006253589440496640027832611063390821766511707...,272.0,1262.0,0,0,0.0,0,1266,0.0,11380.0,-1.0,0,0,6.0,70.0,1266,0,0,17,34,8,2,0
2,20eea5b22fc76d0c1a802b0b43e37135121c19ba3235fa...,57,23759,1386128939984345883611159076460996037107150094...,2957621807105112347001786700780013206935156281...,5,5.0,1295656580959528195878890317272517510574361137...,3396357932789603274781563012123409452058004868...,272.0,3024.0,1,0,0.0,0,3027,42.0,12381.0,-1.0,0,0,6.0,70.0,3027,0,0,2,37,59,1,1
3,72ecbb06661cd74ae4f26828feef5e2ad5da884cbce851...,56,113358,9266034733120664363656227825632130684606573534...,2692235992922997015670407420715992764860752243...,5,5.0,1043994063244910127941407210589116601102921018...,1006253589440496640027832611063390821766511707...,276.0,820.0,1,0,0.0,0,819,0.0,15759.0,-1.0,0,0,6.0,70.0,819,0,0,11,33,58,0,0
4,6123106d4e8e712ec4cc8dcb483a92607c72e939bcd77b...,59,135826,7116076958514668873675520613084165528984827318...,6501428506525536389995197547809427192456469505...,5,4.0,2764270327677967978435929459491199409838445234...,9694665056880918802914193734853718239332787519...,288.0,1614.0,0,0,0.0,0,1620,0.0,15759.0,-1.0,0,0,6.0,70.0,1620,0,0,13,58,26,3,0


In [22]:
df_positive.describe()

,locdt,loctm,contp,etymd,mcc,conam,ecfg,insfg,iterm,bnsfg,...,hcefg,csmcu,csmam,flg_3dsmk,label,loctm_hh,loctm_mm,loctm_ss,weekday,conam3000
count,34213.000000,34213.000000,34213.000000,34140.000000,34210.000000,3.421300e+04,34213.000000,34213.000000,34213.00000,34213.000000,...,34023.000000,33791.000000,3.421300e+04,34213.000000,34213.0,34213.000000,34213.000000,34213.000000,34213.000000,34213.000000
mean,29.938211,129295.684301,4.929121,5.186936,330.896434,5.003352e+03,0.870985,0.000438,0.00491,0.000029,...,5.760544,65.498446,2.872318e+04,0.027358,1.0,12.637740,28.888493,29.434630,2.775232,0.206062
std,17.451603,74491.812566,0.260695,2.681076,68.729280,2.197956e+04,0.335222,0.020934,0.27806,0.005406,...,1.178329,15.824453,1.065455e+06,0.163127,0.0,7.444129,17.325229,17.262945,1.974892,0.404481
min,0.000000,7.000000,2.000000,0.000000,1.000000,0.000000e+00,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,14.000000,60743.000000,5.000000,4.000000,317.000000,1.000000e+02,1.000000,0.000000,0.00000,0.000000,...,6.000000,68.000000,3.300000e+01,0.000000,1.0,6.000000,14.000000,14.000000,1.000000,0.000000
50%,30.000000,143215.000000,5.000000,5.000000,324.000000,1.264570e+03,1.000000,0.000000,0.00000,0.000000,...,6.000000,70.000000,3.990000e+02,0.000000,1.0,14.000000,29.000000,29.000000,3.000000,0.000000
75%,45.000000,194315.000000,5.000000,8.000000,365.000000,2.990000e+03,1.000000,0.000000,0.00000,0.000000,...,6.000000,70.000000,1.991000e+03,0.000000,1.0,19.000000,44.000000,44.000000,4.000000,0.000000
max,59.000000,235957.000000,6.000000,10.000000,500.000000,2.555812e+06,1.000000,1.000000,24.00000,1.000000,...,10.000000,85.000000,7.473480e+07,1.000000,1.0,23.000000,59.000000,59.000000,6.000000,1.000000


In [7]:
# 列出包含NaN值的列名 並替換
nan_columns = data.isnull().any()
columns_with_nan = nan_columns[nan_columns].index.tolist()
print(columns_with_nan)
data[columns_with_nan] = data[columns_with_nan].fillna(-1)

['etymd', 'mcc', 'stocn', 'scity', 'stscd', 'hcefg', 'csmcu']


In [8]:
X = data.drop(['label','txkey'], axis=1) 
y = data['label']
# Step 5 特徵選擇
selector = SelectKBest(f_classif, k="all")  # Change k value as needed 
X_new = selector.fit_transform(X, y)
# 獲取選擇的特徵索引
selected_feature_indices = selector.get_support(indices=True)
# 獲取選擇的特徵名稱
selected_feature_names = X.columns[selected_feature_indices]

In [9]:
f_scores = selector.scores_
for feature_name, f_score in zip(selected_feature_names, f_scores):
    print(f"Feature: {feature_name}, F-score: {f_score}")

Feature: locdt, F-score: 25.732720707422157
Feature: loctm, F-score: 2246.212048769775
Feature: chid, F-score: 67.98372188681121
Feature: cano, F-score: 2.1678964497214155
Feature: contp, F-score: 89.86191045807381
Feature: etymd, F-score: 2099.7801891478725
Feature: mchno, F-score: 3622.4818822921243
Feature: acqic, F-score: 8890.879309787448
Feature: mcc, F-score: 1335.0602606964885
Feature: conam, F-score: 4695.318631813367
Feature: ecfg, F-score: 12407.180690100618
Feature: insfg, F-score: 340.23096985088426
Feature: iterm, F-score: 199.74065404429842
Feature: bnsfg, F-score: 42.4869143094996
Feature: flam1, F-score: 4699.231085728367
Feature: stocn, F-score: 258605.04228082276
Feature: scity, F-score: 30475.308361969375
Feature: stscd, F-score: 1576023.8733040548
Feature: ovrlt, F-score: 2219.0180670710456
Feature: flbmk, F-score: 1.099339555148631
Feature: hcefg, F-score: 2473.0186553035924
Feature: csmcu, F-score: 96.94452738684194
Feature: csmam, F-score: 1888.0609285778685
Fea

In [10]:
# Step 6 切割訓練測試集
# Assuming X_new and y are the selected features and labels after feature selection
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=15)

# Step 7 特徵正規化，方便訓練並降低不同訓練資料特徵維度的比例
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

gamma越小越容易Overfitting,0

Baseline XGB Model

In [11]:
# model = XGBClassifier(max_depth=20, 
#                       learning_rate=0.03,
#                      min_child_weight=7.734890981711013,
#                      scale_pos_weight=5, 
#                      gamma=0,
#                      n_estimators=200, 
#                      random_state=42, 
#                      eval_metric="auc",
#                      tree_method='hist',  # 使用"hist"
#                      device='cuda',  # 使用"cuda"
#                     colsample_bytree=0.7,
#                      colsample_bylevel=1,
#                     objective='binary:logistic')
model = XGBClassifier(max_depth=20, 
                    min_child_weight=5,
                    scale_pos_weight=5, 
                    n_estimators=200, 
                    gamma=0,
                    eval_metric="auc",
                    tree_method='hist',  # 使用"hist"
                    # max_delta_step =1,
                    # colsample_bytree=0.7,
                    # colsample_bylevel=1,
                    seed=1440,
                    device='cuda')  # 使用"cuda"
# model = xgb.XGBClassifier(
#     max_depth=8,
#     learning_rate=0.03,
#     n_estimators=500, 
#     scale_pos_weight=5, 
#     silent=True,
#     objective='binary:logistic',
#     nthread=-1,
#     gamma=0,
#     min_child_weight=0,
#     subsample=0.85,
#     colsample_bytree=0.7,
#     colsample_bylevel=1,
#     scoring='roc_auc',                  
#     seed=1440,
#     tree_method='hist',  # 使用"hist"
#     device='cuda'
#     # missing=None
#     )

# model.fit(X_train, y_train)
print("CV score of XGB is ",cross_val_score(model,X_train,y_train,cv=4, scoring = 'f1').mean())

# Step 9: Model Evaluation
# y_trainpred = model.predict(X_train)
# y_pred = model.predict(X_test)

# print("TrainingData Classification Report: ")
# print(classification_report(y_train, y_trainpred))
# print(confusion_matrix(y_train, y_trainpred))
# print('*'*20)
# print("y_pred Classification Report: ")
# print(classification_report(y_test, y_pred))
# print("Confusion Matrix: ")
# print(confusion_matrix(y_test, y_pred))

c:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:21:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


CV score of XGB is  0.8064206223525281


In [12]:
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print("y_pred Classification Report: ")
print(classification_report(y_test, y_pred))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

y_pred Classification Report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1852570
           1       0.92      0.76      0.83      6944

    accuracy                           1.00   1859514
   macro avg       0.96      0.88      0.92   1859514
weighted avg       1.00      1.00      1.00   1859514

Confusion Matrix: 
[[1852124     446]
 [   1678    5266]]


Baseline LGBM Model

In [39]:
from lightgbm import LGBMClassifier
lgbmmodel = LGBMClassifier(random_state=42,
                        n_estimators = 500,
                        # scale_pos_weight=2,
                        max_depth = 10,
                        learning_rate =  0.025,
                         boosting_type = 'gbdt',
                        device="gpu")
print("CV score of LGBM is ",cross_val_score(lgbmmodel,X_train,y_train,cv=4, scoring = 'f1').mean())

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 20452, number of negative: 5558087
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2687
[LightGBM] [Info] Number of data points in the train set: 5578539, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (106.40 MB) transferred to GPU in 0.138233 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003666 -> initscore=-5.604929
[LightGBM] [Info] Start training from score -5.604929
[L

In [40]:
lgbmmodel.fit(X_train,y_train)
y_pred = lgbmmodel.predict(X_test)

print("y_pred Classification Report: ")
print(classification_report(y_test, y_pred))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 27269, number of negative: 7410783
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2692
[LightGBM] [Info] Number of data points in the train set: 7438052, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (141.87 MB) transferred to GPU in 0.166771 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003666 -> initscore=-5.604941
[LightGBM] [Info] Start training from score -5.604941
[L

Baseline DART Model

In [31]:
dartmodel = LGBMClassifier(random_state = 42, device="gpu", boosting_type = 'dart')
print("CV score of DART is ",cross_val_score(dartmodel,X_train,y_train,cv=4, scoring = 'roc_auc').mean())

[LightGBM] [Info] Number of positive: 20452, number of negative: 5558087
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2687
[LightGBM] [Info] Number of data points in the train set: 5578539, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (106.40 MB) transferred to GPU in 0.104377 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003666 -> initscore=-5.604929
[LightGBM] [Info] Start training from score -5.604929
[LightGBM] [Info] Number of positive: 20452, number of negative: 5558087
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2679
[LightGBM] [Info] Number of data points in the train set: 5578539, number of used features: 29
[LightGBM] [Info

In [34]:
dartmodel.fit(X_train,y_train)
y_pred = dartmodel.predict(X_test)

[LightGBM] [Info] Number of positive: 27269, number of negative: 7410783
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2692
[LightGBM] [Info] Number of data points in the train set: 7438052, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (141.87 MB) transferred to GPU in 0.127994 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003666 -> initscore=-5.604941
[LightGBM] [Info] Start training from score -5.604941


In [35]:
print("y_pred Classification Report: ")
print(classification_report(y_test, y_pred))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

y_pred Classification Report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1852570
           1       0.68      0.36      0.47      6944

    accuracy                           1.00   1859514
   macro avg       0.84      0.68      0.73   1859514
weighted avg       1.00      1.00      1.00   1859514

Confusion Matrix: 
[[1851417    1153]
 [   4456    2488]]


Soft voting

In [41]:
from sklearn.ensemble import VotingClassifier
vcmodel = VotingClassifier([("lgbm",lgbmmodel),("xgb",model),("dart",dartmodel)],voting="soft",weights=[1, 3, 2])
print("CV score of VC is ",cross_val_score(vcmodel,X_train,y_train,cv=4, scoring = 'roc_auc').mean())

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 20452, number of negative: 5558087
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2687
[LightGBM] [Info] Number of data points in the train set: 5578539, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (106.40 MB) transferred to GPU in 0.091036 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003666 -> initscore=-5.604929
[LightGBM] [Info] Start training from score -5.604929
[L

In [42]:
vcmodel.fit(X_train,y_train)
y_pred = vcmodel.predict(X_test)

# print("TrainingData Classification Report: ")
# print(classification_report(y_train, y_trainpred))
# print(confusion_matrix(y_train, y_trainpred))
# print('*'*20)


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 27269, number of negative: 7410783
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2692
[LightGBM] [Info] Number of data points in the train set: 7438052, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (141.87 MB) transferred to GPU in 0.130500 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003666 -> initscore=-5.604941
[LightGBM] [Info] Start training from score -5.604941
[L

In [43]:
print("y_pred Classification Report: ")
print(classification_report(y_test, y_pred))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred))

y_pred Classification Report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1852570
           1       0.91      0.65      0.76      6944

    accuracy                           1.00   1859514
   macro avg       0.95      0.83      0.88   1859514
weighted avg       1.00      1.00      1.00   1859514

Confusion Matrix: 
[[1852097     473]
 [   2418    4526]]


# testing

In [35]:
public1 = pd.read_csv(r"public_processed.csv")
public2 = pd.read_csv(r"private_1_processed.csv")
public_processed_df = pd.concat([public1, public2])

public_feature = public_processed_df.drop(['txkey'], axis=1)
public_name = public_processed_df['txkey']

public_feature["chid"] = public_feature["chid"].apply(hash_to_int)
public_feature["cano"] = public_feature["cano"].apply(hash_to_int)
public_feature["mchno"]= public_feature["mchno"].apply(hash_to_int)
public_feature["acqic"]= public_feature["acqic"].apply(hash_to_int)

In [37]:
public_feature['loctm_hh'] = public_feature['loctm'].apply(lambda x: math.floor(x/10000))
public_feature['loctm_mm'] = public_feature['loctm'].apply(lambda x: math.floor(x/100)-math.floor(x/10000)*100)
public_feature['loctm_ss'] = public_feature['loctm'].apply(lambda x: math.floor(x)-math.floor(x/100)*100)
public_feature['weekday'] = public_feature['locdt'] % 7
public_feature['conam3000'] = np.where(public_feature['conam'] > 3000, 1, 0)

In [38]:
#  自己設計的資料清理
public_feature_nan_columns = public_feature.isnull().any()
public_feature_columns_with_nan = public_feature_nan_columns[public_feature_nan_columns].index.tolist()
print(public_feature_columns_with_nan)
public_feature[public_feature_columns_with_nan] = public_feature[public_feature_columns_with_nan].fillna(-1)
public_training_5 = public_feature[selected_feature_names]

['etymd', 'mcc', 'stocn', 'scity', 'stscd', 'hcefg', 'csmcu']


In [42]:
#  使用訓練集的sc進行縮放
public_X_new = sc.transform(public_training_5)

# 使用上面的xgbc測試集辨識
public_y_pred = model.predict(public_X_new)

# 製作與儲存可以上傳的csv檔案
public_y_pred = pd.DataFrame(public_y_pred, columns=["pred"])

public_y_pred = public_y_pred.reset_index(drop=True)
public_name = public_name.reset_index(drop=True)

result = pd.concat([public_name, public_y_pred], axis=1)
result.to_csv(r"result.csv",index=False)

c:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
